In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch.utils.data import Sampler

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy import stats
import itertools

In [ ]:
import random
random.seed(42)

import torch
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

## Define Dataset

In [ ]:
# Function to assign source names to the dataset equally
def assign_sources_equally(dataset, sources=('A', 'B', 'C', 'D', 'E', 'F')):
    num_sources = len(sources)
    num_data = len(dataset)
    num_each = num_data // num_sources
    
    # Convert sources to a list if it's a tuple
    sources_list = list(sources)
    
    # Create a list of source labels, each repeated equally
    source_labels = sources_list * num_each + [sources_list[i] for i in range(num_data % num_sources)]
    
    # Shuffle the labels to randomize their order
    np.random.shuffle(source_labels)
    
    return source_labels

# Assign sources to training and test data
training_sources = assign_sources_equally(training_data)
test_sources = assign_sources_equally(test_data)

# Example of how you can use the assigned sources
print("First 10 source labels for training data:", training_sources[:10])

In [ ]:
# Function to assign participants to train and test datasets
def assign_participants_to_train_and_test(train_labels, test_labels, num_train_participants_per_source, train_images_per_participant, num_test_participants_per_source):
    train_participants = {source: [] for source in set(train_labels)}
    test_participants = {source: [] for source in set(test_labels)}
    
    # Calculate number of test images for participants
    num_test_participants_total = num_test_participants_per_source * len(test_participants)
    test_images_per_participant = 10000 // num_test_participants_total
    extra_test_images = 10000 % num_test_participants_total

    # Assign participants for training
    global_participant_id = 0
    for source in train_participants:
        train_indices = [i for i, s in enumerate(train_labels) if s == source]
        np.random.shuffle(train_indices)
        
        for i in range(num_train_participants_per_source):
            train_participant_indices = train_indices[i * train_images_per_participant: (i + 1) * train_images_per_participant]
            train_participants[source].append((global_participant_id, train_participant_indices))
            global_participant_id += 1

    # Assign participants for testing
    for source in test_participants:
        test_indices = [i for i, s in enumerate(test_labels) if s == source]
        np.random.shuffle(test_indices)
        
        for i in range(num_test_participants_per_source):
            if i < extra_test_images:
                test_participant_indices = test_indices[i * (test_images_per_participant + 1): (i + 1) * (test_images_per_participant + 1)]
            else:
                test_participant_indices = test_indices[i * test_images_per_participant: (i + 1) * test_images_per_participant]
            test_participants[source].append((global_participant_id, test_participant_indices))
            global_participant_id += 1

    return train_participants, test_participants

def count_unique_participants(participants):
    unique_participants = set()
    for source, participant_info in participants.items():
        for participant_id, _ in participant_info:
            unique_participants.add(participant_id)
    return len(unique_participants)

# Example data
training_data_indices = np.arange(60000)
test_data_indices = np.arange(10000)

# Assign sources to training and test data
training_sources = assign_sources_equally(training_data_indices)
test_sources = assign_sources_equally(test_data_indices)

# Parameters
num_train_participants_per_source = 10
num_test_participants_per_source = 2
train_images_per_participant = 1000

# Assign participant IDs to both training and test data
train_participants, test_participants = assign_participants_to_train_and_test(
    training_sources, test_sources, num_train_participants_per_source, train_images_per_participant, num_test_participants_per_source)

# Count unique participants in training and test datasets separately
num_unique_train_participants = count_unique_participants(train_participants)
num_unique_test_participants = count_unique_participants(test_participants)

print(f"Number of unique participants in training data: {num_unique_train_participants}")
print(f"Number of unique participants in test data: {num_unique_test_participants}")

# Example of how you can use the assigned participants
for source, participant_info in train_participants.items():
    print(f"Source: {source}")
    for participant_id, train_indices in participant_info:
        print(f"  Participant {participant_id} - Train Indices: {train_indices[:10]}...")

for source, participant_info in test_participants.items():
    print(f"Source: {source}")
    for participant_id, test_indices in participant_info:
        print(f"  Participant {participant_id} - Test Indices: {test_indices[:10]}...")

In [ ]:
# UPDATED: Create custom dataset class
class ParticipantCustomMNIST(Dataset):
    def __init__(self, mnist_dataset, source_labels, participants, transform=None):
        self.mnist_dataset = mnist_dataset
        self.source_labels = source_labels
        self.participants = participants
        self.transform = transform

        self.data = self._create_data()

    def _create_data(self):
        data = []
        for source, participant_data in self.participants.items():
            for participant_id, indices in participant_data:
                for idx in indices:
                    image, label = self.mnist_dataset[idx]  # Access original MNIST data
                    source_label = self.source_labels[idx]
                    data.append((image, label, source_label, participant_id))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label, source_label, participant_id = self.data[idx]

        if source_label == 'A':
            image = transforms.functional.rotate(image, 180)
        elif source_label == 'B':
            c, h, w = image.shape
            num_pixels = h * w
            num_missing = num_pixels // 2
            mask = torch.randperm(num_pixels)[:num_missing]
            mask_h, mask_w = mask // w, mask % w
            image[:, mask_h, mask_w] = 1
        elif source_label == 'C':
            noise = torch.randn_like(image) * 0.5
            image = image + noise
            image = torch.clamp(image, 0, 1)
        elif source_label == 'D':
            label_permutation = {0: 9, 1: 8, 2: 7, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1, 9: 0}
            label = label_permutation[label]
        elif source_label == 'E':
            pass
        elif source_label == 'F':
            pass
        else:
            raise ValueError("Unknown source label provided: must be 'A', 'B', 'C', 'D', or 'E', or 'F'")

        if self.transform:
            image = self.transform(image)

        image = torch.flatten(image)
        return image, label, source_label, participant_id

In [ ]:
# Create custom datasets with participants
train_dataset = ParticipantCustomMNIST(training_data, training_sources, train_participants, transform=None)
test_dataset = ParticipantCustomMNIST(test_data, test_sources, test_participants, transform=None)

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
for images, labels, source_labels, participant_ids in train_loader:
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

    print("Participant IDs shape:", participant_ids.shape)
    break

In [ ]:
train_dataset.source_labels[:10]

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleMLP, self).__init__()
        self.linear1 = nn.Linear(input_size, 100)
        self.linear2 = nn.Linear(100,100)
        self.linear3 = nn.Linear(100, output_size)
    
    def forward(self, data):
        x = data
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        y_pred = self.linear3(x)
        return y_pred

In [ ]:
model = SimpleMLP(input_size=784, output_size=10)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=0.001,
    betas=(0.9,0.999)
    )

In [ ]:
print(model)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs, target_epoch=3):
    # Set the model to training mode
    model.train()
    
    # List to store individual participant losses for the target epoch
    target_epoch_participant_losses = []
    

    # Dictionary to store epoch losses for each source
    epoch_losses = {'A': [], 'B': [], 'C': [], 'D': [], 'E': [], 'F': []}

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        # Initialize source-specific counters
        source_correct = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
        source_total = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
        source_running_loss = {'A': 0.0, 'B': 0.0, 'C': 0.0, 'D': 0.0, 'E': 0.0, 'F': 0.0}

        # Initialize participant losses
        participant_losses = {participant_id: [] for participant_id in range(60)}

        for batch_idx, (images, labels, source_labels, participant_ids) in enumerate(train_loader):
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += loss.item() * images.size(0)

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            # Update source-specific counters and participant losses
            for i, (source, pred, label, participant_id) in enumerate(zip(source_labels, predicted, labels, participant_ids)):
                individual_loss = criterion(outputs[i].unsqueeze(0), label.unsqueeze(0)).item()
                if epoch == target_epoch - 1:
                    participant_losses[participant_id.item()].append(individual_loss)
                source_running_loss[source] += individual_loss
                
                if pred == label:
                    source_correct[source] += 1
                source_total[source] += 1

        if epoch == target_epoch - 1:
            # Calculate average loss per participant
            for participant_id in participant_losses:
                if participant_losses[participant_id]:  # Avoid division by zero
                    avg_loss = sum(participant_losses[participant_id]) / len(participant_losses[participant_id])
                    target_epoch_participant_losses.append(avg_loss)

            # Save the model state at the target epoch
            target_epoch_model_state = model.state_dict()


        # Calculate epoch loss and overall accuracy
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = (correct_predictions / total_predictions) * 100

        for source in source_running_loss:
            if source_total[source] > 0:
                epoch_source_loss = source_running_loss[source] / source_total[source]
                epoch_losses[source].append(epoch_source_loss)
                source_running_loss[source] = 0  # Reset running loss after calculation

        # Display overall and source-specific accuracies
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
        for source in source_correct:
            if source_total[source] > 0:
                source_accuracy = (source_correct[source] / source_total[source]) * 100
                print(f'Accuracy for Source {source}: {source_accuracy:.2f}%')

    return target_epoch_participant_losses, epoch_losses, target_epoch_model_state

target_epoch_participant_losses, epoch_losses, target_epoch_model_state = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10)

In [ ]:
def evaluate_model_at_epoch(model, test_loader, criterion, target_epoch_model_state):
    model.load_state_dict(target_epoch_model_state)  # Load the saved model state
    model.eval()  # Set the model to evaluation mode
    
    # Initialize an empty dictionary for test participant losses
    test_participant_losses = {}

    with torch.no_grad():
        for batch_idx, (images, labels, source_labels, participant_ids) in enumerate(test_loader):
            outputs = model(images)
            for i in range(len(labels)):
                participant_id = participant_ids[i].item()
                if participant_id not in test_participant_losses:
                    test_participant_losses[participant_id] = []
                individual_loss = criterion(outputs[i].unsqueeze(0), labels[i].unsqueeze(0)).item()
                test_participant_losses[participant_id].append(individual_loss)

    # Calculate average loss per test participant
    target_epoch_test_participant_losses = []
    for participant_id in test_participant_losses:
        if test_participant_losses[participant_id]:  # Avoid division by zero
            avg_loss = sum(test_participant_losses[participant_id]) / len(test_participant_losses[participant_id])
            target_epoch_test_participant_losses.append(avg_loss)

    return target_epoch_test_participant_losses

# Example call to the evaluation function
target_epoch_test_participant_losses = evaluate_model_at_epoch(model, test_loader, criterion, target_epoch_model_state)

# Print the results
print("Test participants' average losses at target epoch:")
for participant_id, avg_loss in enumerate(target_epoch_test_participant_losses, start=72):
    print(f"Participant {participant_id}: Average Loss = {avg_loss:.4f}")

In [ ]:
unique_train_participants = len(set(target_epoch_participant_losses))
print(f"Number of unique participants in training losses: {unique_train_participants}")

In [ ]:
epoch_losses['E'] = [(e + f) / 2 for e, f in zip(epoch_losses['E'], epoch_losses['F'])]
del epoch_losses['F']  # Remove the old 'F' data

# Plotting the results
for source in epoch_losses:
    plt.plot(range(1, len(epoch_losses[source]) + 1), epoch_losses[source], marker='o', label=f'Source {source}')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epoch Losses Over Time for Each Source')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Colors to use for the plots
colors = ['deepskyblue', 'teal', 'mediumblue', 'darkturquoise', 'aqua']

# Plotting the results
for idx, (source, color) in enumerate(zip(epoch_losses, colors)):
    plt.plot(range(1, len(epoch_losses[source]) + 1), epoch_losses[source], marker='o', color=color, label=f'Source {source}')
plt.xlabel('Epoch')
plt.ylabel('Loss')
# Title removed as per your request

# Remove the top and right spines
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add grey dashed grid lines
plt.grid(True, linestyle='--', color='grey')

# Add a vertical line at epoch 3
plt.axvline(x=3, color='orchid', linestyle='--', linewidth=1.5)

# Add a horizontal legend at the top, moved up a bit
plt.legend(loc='upper center', ncol=len(epoch_losses), bbox_to_anchor=(0.5, 1.10))

plt.show()

In [ ]:

plt.figure()
plt.hist(target_epoch_participant_losses, bins=30, alpha=0.7)
plt.xlabel('Average Loss')
plt.ylabel('Frequency')
plt.title(f'Average Loss Distribution for Participants at Epoch 3')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

# Assuming target_epoch_losses is your list of individual losses at the second epoch
losses = np.array(target_epoch_participant_losses).reshape(-1, 1)

# Use the Elbow Method to find the optimal number of clusters
wcss = []
for i in range(1, 11):  # Test for 1 to 10 clusters
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(losses)
    wcss.append(kmeans.inertia_)


In [ ]:
# Plot the Elbow graph
plt.figure()
plt.plot(range(1, 11), wcss, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')

# Remove the top and right spines
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add dashed grid lines
plt.grid(True, linestyle='--', color='grey')

# Add a vertical line at epoch 3
plt.axvline(x=3, color='orchid', linestyle='--', linewidth=1.5)

plt.show()

In [ ]:
train_losses = np.array(target_epoch_participant_losses).reshape(-1, 1)

# Choose the number of clusters (e.g., 3)
num_clusters = 3

# Fit K-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
train_clusters = kmeans.fit_predict(train_losses)

# Predict clusters for the test losses
test_losses = np.array(target_epoch_test_participant_losses).reshape(-1, 1)
test_clusters = kmeans.predict(test_losses)

# Plot the histogram with cluster information
for cluster in range(num_clusters):
    cluster_losses = train_losses[train_clusters == cluster]
    plt.hist(cluster_losses, bins=30, alpha=0.7, label=f'Cluster {cluster + 1}')

plt.xlabel('Loss')
plt.ylabel('Frequency')
plt.title('Loss Distribution for Each Cluster at Epoch 3')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Colors to use for the histograms
colors = ['lightseagreen', 'cornflowerblue', 'slateblue']

# Plot the histogram with cluster information
for cluster, color in zip(range(num_clusters), colors):
    cluster_losses = train_losses[train_clusters == cluster]
    plt.hist(cluster_losses, bins=30, alpha=0.7, color=color, label=f'Cluster {cluster + 1}')

plt.xlabel('Loss')
plt.ylabel('Number of Participants')
# Title removed as per your request

# Remove the top and right spines
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add grey dashed grid lines
plt.grid(True, linestyle='--', color='grey')

# Add a horizontal legend at the top, moved up a bit
plt.legend(loc='upper center', ncol=num_clusters, bbox_to_anchor=(0.5, 1.15))

plt.show()

In [ ]:
num_clusters = 3  # Number of clusters

# Colors to use for the histograms
colors = ['lightseagreen', 'cornflowerblue', 'slateblue']

# Plot the histogram with cluster information
plt.figure(figsize=(8, 6))
for cluster, color in zip(range(num_clusters), colors):
    cluster_losses = train_losses[train_clusters == cluster]
    plt.hist(cluster_losses, bins=30, alpha=0.7, color=color, label=f'Cluster {cluster + 1}')

plt.xlabel('Loss')
plt.ylabel('Number of Participants')
# Title removed as per your request

# Remove the top and right spines
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add grey dashed grid lines
plt.grid(True, linestyle='--', color='grey')

# Add a horizontal legend at the top, moved up a bit
plt.legend(loc='upper center', ncol=num_clusters, bbox_to_anchor=(0.5, 1.10))

plt.show()

# Display the number of participants in each cluster
for cluster in range(num_clusters):
    num_in_cluster = sum(train_clusters == cluster)
    print(f'Number of participants in Cluster {cluster + 1}: {num_in_cluster}')

In [ ]:
# Display the number of participants in Cluster 1
num_in_cluster_1 = sum(train_clusters == 0)  # Assuming clusters are zero-indexed
print(f'Number of participants in Cluster 1: {num_in_cluster_1}')


In [ ]:
# Create the mapping for train participants
train_participant_cluster_mapping = {participant_id: cluster for participant_id, cluster in enumerate(train_clusters)}

# Create the mapping for test participants
start_test_participant_id = len(train_clusters)  # Continue the participant IDs from where train ends
test_participant_cluster_mapping = {start_test_participant_id + participant_id: cluster for participant_id, cluster in enumerate(test_clusters)}

# Combine both mappings
participant_cluster_mapping = {**train_participant_cluster_mapping, **test_participant_cluster_mapping}

# Print the combined mapping
print("\nParticipant ID to Cluster mapping:")
for participant_id, cluster in participant_cluster_mapping.items():
    print(f"Participant {participant_id}: Cluster {cluster}")

In [ ]:
# Extract source labels and participant IDs from the train_dataset
source_labels_dict = {}
for i in range(len(train_dataset)):
    _, _, source_label, participant_id = train_dataset[i]
    if participant_id not in source_labels_dict:
        source_labels_dict[participant_id] = source_label

# Extract source labels and participant IDs from the test_dataset
for i in range(len(test_dataset)):
    _, _, source_label, participant_id = test_dataset[i]
    if participant_id not in source_labels_dict:
        source_labels_dict[participant_id] = source_label

# Verify mapping
print("Participant ID to Source Label mapping:")
for participant_id, source_label in source_labels_dict.items():
    print(f"Participant {participant_id}: Source {source_label}")

# Create source_labels list from the dictionary
source_labels = [source_labels_dict[participant_id] for participant_id in sorted(source_labels_dict.keys())]

# Print the source_labels list
print("\nSource Labels list:")
print(source_labels)

In [ ]:
# Check for missing participant IDs
missing_keys = [pid for pid in participant_cluster_mapping.keys() if pid not in source_labels_dict]
print(f"Missing participant IDs in source_labels_dict: {missing_keys}")

In [ ]:
# Create a DataFrame for visualization
data = {
    'Participant ID': list(participant_cluster_mapping.keys()),
    'Cluster': list(participant_cluster_mapping.values()),
    'Source': [source_labels_dict[pid] for pid in participant_cluster_mapping.keys()]
}

df = pd.DataFrame(data)

# Create a contingency table
contingency_table = pd.crosstab(df['Source'], df['Cluster'])

# Visualize using a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(contingency_table, annot=True, fmt='d', cmap='Blues')
plt.title('Heatmap of Source Labels vs Clusters')
plt.ylabel('Source')
plt.xlabel('Cluster')
plt.show()

In [ ]:
# Sample data and participant_cluster_mapping and source_labels_dict are assumed to be defined
data = {
    'Participant ID': list(participant_cluster_mapping.keys()),
    'Cluster': list(participant_cluster_mapping.values()),
    'Source': [source_labels_dict[pid] for pid in participant_cluster_mapping.keys()]
}

df = pd.DataFrame(data)

# Combine 'E' and 'F' sources into 'E'
df['Source'] = df['Source'].replace('F', 'E')

# Split the DataFrame into two based on Participant ID ranges
df1 = df[df['Participant ID'].isin(range(1, 61))]
df2 = df[df['Participant ID'].isin(range(60, 73))]

# Create contingency tables for both DataFrames
contingency_table1 = pd.crosstab(df1['Cluster'], df1['Source'])
contingency_table2 = pd.crosstab(df2['Cluster'], df2['Source'])

# Ensure all clusters are displayed, even if they have zero data points
for cluster in range(1, 3):
    if cluster not in contingency_table2.index:
        contingency_table2.loc[cluster] = 0
contingency_table2 = contingency_table2.sort_index()

# Visualize using heatmaps

# First heatmap
plt.figure(figsize=(7, 6))
ax1 = sns.heatmap(contingency_table1, annot=True, fmt='d', cmap='Blues', cbar=True, cbar_kws={'shrink': 0.75})
ax1.set_ylabel('Cluster')
ax1.set_xlabel('Source')
ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)  # Ensure labels are horizontal

# Manually set y-tick labels to start from 1
ax1.set_yticks(ax1.get_yticks())
ax1.set_yticklabels(range(1, len(ax1.get_yticks()) + 1))

plt.tight_layout()
plt.show()

# Second heatmap
plt.figure(figsize=(7, 6))
ax2 = sns.heatmap(contingency_table2, annot=True, fmt='d', cmap='Blues', cbar=True, cbar_kws={'shrink': 0.75})
ax2.set_ylabel('Cluster')
ax2.set_xlabel('Source')
ax2.set_yticklabels(ax2.get_yticklabels(), rotation=0)  # Ensure labels are horizontal

# Manually set y-tick labels to start from 1
ax2.set_yticks(ax2.get_yticks())
ax2.set_yticklabels(range(1, len(ax2.get_yticks()) + 1))

plt.tight_layout()
plt.show()

In [ ]:
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    # Initialize source-specific counters
    source_correct = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
    source_total = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
    source_running_loss = {'A': 0.0, 'B': 0.0, 'C': 0.0, 'D': 0.0, 'E': 0.0, 'F': 0.0}

    # Initialize participant losses
    participant_losses = {}

    with torch.no_grad():
        for batch_idx, (images, labels, source_labels, participant_ids) in enumerate(test_loader):
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update source-specific counters and participant losses
            for i, (source, pred, label, participant_id) in enumerate(zip(source_labels, predicted, labels, participant_ids)):
                individual_loss = criterion(outputs[i].unsqueeze(0), label.unsqueeze(0)).item()
                if participant_id.item() not in participant_losses:
                    participant_losses[participant_id.item()] = []
                participant_losses[participant_id.item()].append(individual_loss)
                source_running_loss[source] += individual_loss

                if pred == label:
                    source_correct[source] += 1
                source_total[source] += 1

    accuracy = 100 * correct / total
    print(f'Accuracy on the entire test set: {accuracy:.2f}%')

    # Display source-specific accuracies
    for source in source_correct:
        if source_total[source] > 0:
            source_accuracy = (source_correct[source] / source_total[source]) * 100
            print(f'Accuracy for Source {source}: {source_accuracy:.2f}%')

    # Calculate average loss per participant
    participant_avg_losses = []
    for participant_id in participant_losses:
        if participant_losses[participant_id]:  # Avoid division by zero
            avg_loss = sum(participant_losses[participant_id]) / len(participant_losses[participant_id])
            participant_avg_losses.append(avg_loss)

    return participant_avg_losses

# Evaluate the model with source-specific accuracies and participant-specific losses
participant_avg_losses = evaluate_model(model, test_loader, criterion)

# Plot the histogram of participant losses
import matplotlib.pyplot as plt

plt.figure()
plt.hist(participant_avg_losses, bins=30, alpha=0.7)
plt.xlabel('Average Loss')
plt.ylabel('Frequency')
plt.title('Average Loss Distribution for Participants in Test Set')
plt.grid(True)
plt.show()

In [ ]:
class CustomMNISTWithCluster(ParticipantCustomMNIST):
    def __init__(self, mnist_dataset, source_labels, participant_ids, participant_cluster_mapping, transform=None):
        super().__init__(mnist_dataset, source_labels, participant_ids, transform)
        self.participant_cluster_mapping = participant_cluster_mapping

    def __getitem__(self, idx):
        image, label, source_label, participant_id = super().__getitem__(idx)
        loss_cluster = self.participant_cluster_mapping[participant_id]
        return image, label, source_label, participant_id, loss_cluster

# Example instantiation
custom_train_dataset_with_cluster = CustomMNISTWithCluster(training_data, training_sources, train_participants, participant_cluster_mapping, transform=None)
custom_test_dataset_with_cluster = CustomMNISTWithCluster(test_data, test_sources, test_participants, participant_cluster_mapping, transform=None)


In [ ]:
train_loader_loss = DataLoader(custom_train_dataset_with_cluster, batch_size=128, shuffle=True)
test_loader_loss = DataLoader(custom_test_dataset_with_cluster, batch_size=128, shuffle=False)

# loss mlp final layer

In [ ]:
class LossMLP(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate):
        super(LossMLP, self).__init__()
        # Common layers for all clusters
        self.linear1 = nn.Linear(input_size, 100)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(100, 100)
        self.dropout2 = nn.Dropout(dropout_rate)

        # Special final layer for each cluster
        self.linear3_cluster1 = nn.Linear(100, output_size)
        self.linear3_cluster2 = nn.Linear(100, output_size)
        self.linear3_cluster3 = nn.Linear(100, output_size)

    def forward(self, data, clusters):
        outputs = []
        for i, cluster in enumerate(clusters):
            x = F.relu(self.linear1(data[i]))
            x = self.dropout1(x)
            x = F.relu(self.linear2(x))
            x = self.dropout2(x)

            # Choose the final layer based on the loss cluster
            if cluster == 0:
                y_pred = self.linear3_cluster1(x)
            elif cluster == 1:
                y_pred = self.linear3_cluster2(x)
            elif cluster == 2:
                y_pred = self.linear3_cluster3(x)
            else:
                print(f"Unexpected cluster: {cluster}")
                raise ValueError("Unknown cluster provided: must be 0, 1, or 2")
            outputs.append(y_pred)
        return torch.stack(outputs)



In [ ]:
def train_model(lossmodel, train_loader, criterion, optimizerloss, num_epochs):
    lossmodel.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        source_correct = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
        source_total = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}

        for batch_idx, (images, labels, source_labels, participant_ids, loss_clusters) in enumerate(train_loader):
            optimizerloss.zero_grad()
            outputs = lossmodel(images, loss_clusters)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizerloss.step()

            running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            for source, pred, label in zip(source_labels, predicted, labels):
                if pred == label:
                    source_correct[source] += 1
                source_total[source] += 1

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = (correct_predictions / total_predictions) * 100

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
        for source in source_correct:
            if source_total[source] > 0:
                source_accuracy = (source_correct[source] / source_total[source]) * 100
                print(f'Accuracy for Source {source}: {source_accuracy:.2f}%')

def validate_model(lossmodel, val_loader, criterion):
    lossmodel.eval()
    val_running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    source_correct = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
    source_total = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}

    with torch.no_grad():
        for batch_idx, (images, labels, source_labels, participant_ids, loss_clusters) in enumerate(val_loader):
            outputs = lossmodel(images, loss_clusters)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            for source, pred, label in zip(source_labels, predicted, labels):
                if pred == label:
                    source_correct[source] += 1
                source_total[source] += 1

    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = correct_predictions / total_predictions * 100.0

    source_accuracies = {}
    for source in source_correct:
        if source_total[source] > 0:
            source_accuracy = (source_correct[source] / source_total[source]) * 100
            source_accuracies[source] = source_accuracy

    return val_loss, val_accuracy, source_accuracies

In [ ]:
# Hyperparameter grid
hyperparameter_grid = {
    'lr': [0.001, 0.005, 0.01],
    'dropout_rate': [0, 0.2, 0.5]
}

best_accuracy = 0
best_model_state = None
best_hyperparams = None

# Create all possible combinations of hyperparameters
all_combinations = list(itertools.product(*hyperparameter_grid.values()))

# Cross-validation and hyperparameter tuning
kf = KFold(n_splits=5, shuffle=True, random_state=42)
criterion = nn.CrossEntropyLoss()

for lr, dropout_rate in all_combinations:
    fold_accuracies = []
    fold_source_accuracies = {'A': [], 'B': [], 'C': [], 'D': [], 'E': [], 'F': []}
    print(f'Testing parameters: lr={lr}, dropout_rate={dropout_rate}')
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(custom_train_dataset_with_cluster)))):
        print(f'Starting Fold {fold+1}')
        train_subset = Subset(custom_train_dataset_with_cluster, train_idx)
        val_subset = Subset(custom_train_dataset_with_cluster, val_idx)
        train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)

        lossmodel = LossMLP(input_size=784, output_size=10, dropout_rate=dropout_rate)
        optimizerloss = torch.optim.Adam(lossmodel.parameters(), lr=lr)

        train_model(lossmodel, train_loader, criterion, optimizerloss, num_epochs=10)
        val_loss, val_acc, source_accuracies = validate_model(lossmodel, val_loader, criterion)
        fold_accuracies.append(val_acc)
        for source in source_accuracies:
            fold_source_accuracies[source].append(source_accuracies[source])

    mean_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_hyperparams = {'lr': lr, 'dropout_rate': dropout_rate}
        best_model = lossmodel

    print(f'Parameters: lr={lr}, dropout_rate={dropout_rate}, Mean Accuracy: {mean_accuracy:.2f}%, Std Dev: {std_accuracy:.2f}%')
    for source in fold_source_accuracies:
        mean_source = np.mean(fold_source_accuracies[source])
        std_source = np.std(fold_source_accuracies[source])
        print(f'Source {source} - Mean: {mean_source:.2f}%, Std Dev: {std_source:.2f}%')

print(f'Best Hyperparameters: {best_hyperparams}, with mean accuracy: {best_accuracy:.2f}%')

In [ ]:
# Bootstrap sampling and testing
def bootstrap_train_and_test(lossmodel, train_data, test_loader, criterion, optimizer_params, num_epochs=10, num_bootstrap=5, sample_percentage=0.8):
    bootstrap_accuracies = []
    source_bootstrap_accuracies = {source: [] for source in 'ABCDEF'}
    for i in range(num_bootstrap):
        # Create a bootstrap sample from the training data
        indices = np.random.choice(len(train_data), size=int(sample_percentage * len(train_data)), replace=True)
        bootstrap_subset = Subset(train_data, indices)
        bootstrap_loader = DataLoader(bootstrap_subset, batch_size=128, shuffle=True)

        # Initialize and train the model
        optimizerloss = torch.optim.Adam(lossmodel.parameters(), **optimizer_params)

        for epoch in range(num_epochs):
            lossmodel.train()
            for images, labels, source_labels, participant_ids, loss_clusters in bootstrap_loader:
                optimizerloss.zero_grad()
                outputs = lossmodel(images, loss_clusters)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizerloss.step()

        # Evaluate on the test set
        lossmodel.eval()
        test_loss, correct, total = 0, 0, 0
        source_counts = {s: 0 for s in 'ABCDEF'}
        source_correct = {s: 0 for s in 'ABCDEF'}
        with torch.no_grad():
            for images, labels, source_labels, participant_ids, loss_clusters in test_loader:
                outputs = lossmodel(images, loss_clusters)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                for i, source in enumerate(source_labels):
                    source_counts[source] += 1
                    if predicted[i] == labels[i]:
                        source_correct[source] += 1

        accuracy = correct / total * 100.0
        bootstrap_accuracies.append(accuracy)
        source_accuracies = {s: (source_correct[s] / source_counts[s] * 100) if source_counts[s] > 0 else 0 for s in 'ABCDEF'}
        for source in source_accuracies:
            source_bootstrap_accuracies[source].append(source_accuracies[source])

    mean_accuracy = np.mean(bootstrap_accuracies)
    std_accuracy = np.std(bootstrap_accuracies)
    mean_source_accuracies = {s: np.mean(source_bootstrap_accuracies[s]) for s in 'ABCDEF'}
    std_source_accuracies = {s: np.std(source_bootstrap_accuracies[s]) for s in 'ABCDEF'}

    return mean_accuracy, std_accuracy, mean_source_accuracies, std_source_accuracies

test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Perform bootstrap training and testing
mean_bootstrap, std_bootstrap, mean_source_accuracies, std_source_accuracies = bootstrap_train_and_test(
    best_model, custom_train_dataset_with_cluster, test_loader_loss, criterion, {'lr': best_hyperparams['lr']}, num_epochs=10, num_bootstrap=5, sample_percentage=0.8
)

print(f'Bootstrap results: Mean accuracy: {mean_bootstrap:.2f}%, Std Dev: {std_bootstrap:.2f}%')
for source in mean_source_accuracies:
    print(f'Source {source} - Bootstrap Mean: {mean_source_accuracies[source]:.2f}%, Std Dev: {std_source_accuracies[source]:.2f}%')

# loss mlp separate

In [ ]:
class LossSeparateMLP(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate):
        super(LossSeparateMLP, self).__init__()
        # Separate layers for source A
        self.linear1_1 = nn.Linear(input_size, 100)
        self.dropout1_1 = nn.Dropout(dropout_rate)
        self.linear2_1 = nn.Linear(100, 100)
        self.dropout2_1 = nn.Dropout(dropout_rate)
        self.linear3_1 = nn.Linear(100, output_size)
        
        # Separate layers for source B
        self.linear1_2 = nn.Linear(input_size, 100)
        self.dropout1_2 = nn.Dropout(dropout_rate)
        self.linear2_2 = nn.Linear(100, 100)
        self.dropout2_2 = nn.Dropout(dropout_rate)
        self.linear3_2 = nn.Linear(100, output_size)
        
        # Separate layers for source C
        self.linear1_3 = nn.Linear(input_size, 100)
        self.dropout1_3 = nn.Dropout(dropout_rate)
        self.linear2_3 = nn.Linear(100, 100)
        self.dropout2_3 = nn.Dropout(dropout_rate)
        self.linear3_3 = nn.Linear(100, output_size)

    def forward(self, data, clusters):
        outputs = []
        for i, cluster in enumerate(clusters):
            if cluster == 0:
                x = F.relu(self.linear1_1(data[i]))
                x = self.dropout1_1(x)
                x = F.relu(self.linear2_1(x))
                x = self.dropout2_1(x)
                y_pred = self.linear3_1(x)
            elif cluster == 1:
                x = F.relu(self.linear1_2(data[i]))
                x = self.dropout1_2(x)
                x = F.relu(self.linear2_2(x))
                x = self.dropout2_2(x)
                y_pred = self.linear3_2(x)
            elif cluster == 2:
                x = F.relu(self.linear1_3(data[i]))
                x = self.dropout1_3(x)
                x = F.relu(self.linear2_3(x))
                x = self.dropout2_3(x)
                y_pred = self.linear3_3(x)
            else:
                print(f"Unexpected cluster: {cluster}")
                raise ValueError("Unknown cluster provided: must be 0, 1, or 2")
            outputs.append(y_pred)
        return torch.stack(outputs)

In [ ]:
def train_model(lossmodel, train_loader, criterion, optimizerloss, num_epochs):
    lossmodel.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        source_correct = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
        source_total = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}

        for batch_idx, (images, labels, source_labels, participant_ids, loss_clusters) in enumerate(train_loader):
            optimizerloss.zero_grad()
            outputs = lossmodel(images, loss_clusters)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizerloss.step()

            running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            for source, pred, label in zip(source_labels, predicted, labels):
                if pred == label:
                    source_correct[source] += 1
                source_total[source] += 1

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = (correct_predictions / total_predictions) * 100

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
        for source in source_correct:
            if source_total[source] > 0:
                source_accuracy = (source_correct[source] / source_total[source]) * 100
                print(f'Accuracy for Source {source}: {source_accuracy:.2f}%')

def validate_model(lossmodel, val_loader, criterion):
    lossmodel.eval()
    val_running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    source_correct = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
    source_total = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}

    with torch.no_grad():
        for batch_idx, (images, labels, source_labels, participant_ids, loss_clusters) in enumerate(val_loader):
            outputs = lossmodel(images, loss_clusters)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            for source, pred, label in zip(source_labels, predicted, labels):
                if pred == label:
                    source_correct[source] += 1
                source_total[source] += 1

    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = correct_predictions / total_predictions * 100.0

    source_accuracies = {}
    for source in source_correct:
        if source_total[source] > 0:
            source_accuracy = (source_correct[source] / source_total[source]) * 100
            source_accuracies[source] = source_accuracy

    return val_loss, val_accuracy, source_accuracies

In [ ]:
# Hyperparameter grid
hyperparameter_grid = {
    'lr': [0.001, 0.005, 0.01],
    'dropout_rate': [0, 0.2, 0.5]
}

best_accuracy = 0
best_model_state = None
best_hyperparams = None

# Create all possible combinations of hyperparameters
all_combinations = list(itertools.product(*hyperparameter_grid.values()))

# Cross-validation and hyperparameter tuning
kf = KFold(n_splits=5, shuffle=True, random_state=42)
criterion = nn.CrossEntropyLoss()

for lr, dropout_rate in all_combinations:
    fold_accuracies = []
    fold_source_accuracies = {'A': [], 'B': [], 'C': [], 'D': [], 'E': [], 'F': []}
    print(f'Testing parameters: lr={lr}, dropout_rate={dropout_rate}')
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(custom_train_dataset_with_cluster)))):
        print(f'Starting Fold {fold+1}')
        train_subset = Subset(custom_train_dataset_with_cluster, train_idx)
        val_subset = Subset(custom_train_dataset_with_cluster, val_idx)
        train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)

        lossmodel = LossSeparateMLP(input_size=784, output_size=10, dropout_rate=dropout_rate)
        optimizerloss = torch.optim.Adam(lossmodel.parameters(), lr=lr)

        train_model(lossmodel, train_loader, criterion, optimizerloss, num_epochs=10)
        val_loss, val_acc, source_accuracies = validate_model(lossmodel, val_loader, criterion)
        fold_accuracies.append(val_acc)
        for source in source_accuracies:
            fold_source_accuracies[source].append(source_accuracies[source])

    mean_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_hyperparams = {'lr': lr, 'dropout_rate': dropout_rate}
        best_model = lossmodel

    print(f'Parameters: lr={lr}, dropout_rate={dropout_rate}, Mean Accuracy: {mean_accuracy:.2f}%, Std Dev: {std_accuracy:.2f}%')
    for source in fold_source_accuracies:
        mean_source = np.mean(fold_source_accuracies[source])
        std_source = np.std(fold_source_accuracies[source])
        print(f'Source {source} - Mean: {mean_source:.2f}%, Std Dev: {std_source:.2f}%')

print(f'Best Hyperparameters: {best_hyperparams}, with mean accuracy: {best_accuracy:.2f}%')

In [ ]:
# Bootstrap sampling and testing
def bootstrap_train_and_test(lossmodel, train_data, test_loader, criterion, optimizer_params, num_epochs=10, num_bootstrap=5, sample_percentage=0.8):
    bootstrap_accuracies = []
    source_bootstrap_accuracies = {source: [] for source in 'ABCDEF'}
    for i in range(num_bootstrap):
        # Create a bootstrap sample from the training data
        indices = np.random.choice(len(train_data), size=int(sample_percentage * len(train_data)), replace=False)
        bootstrap_subset = Subset(train_data, indices)
        bootstrap_loader = DataLoader(bootstrap_subset, batch_size=128, shuffle=True)

        # Initialize and train the model
        optimizerloss = torch.optim.Adam(lossmodel.parameters(), **optimizer_params)

        for epoch in range(num_epochs):
            lossmodel.train()
            for images, labels, source_labels, participant_ids, loss_clusters in bootstrap_loader:
                optimizerloss.zero_grad()
                outputs = lossmodel(images, loss_clusters)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizerloss.step()

        # Evaluate on the test set
        lossmodel.eval()
        test_loss, correct, total = 0, 0, 0
        source_counts = {s: 0 for s in 'ABCDEF'}
        source_correct = {s: 0 for s in 'ABCDEF'}
        with torch.no_grad():
            for images, labels, source_labels, participant_ids, loss_clusters in test_loader:
                outputs = lossmodel(images, loss_clusters)
                loss = criterion(outputs, labels)
                test_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                for i, source in enumerate(source_labels):
                    source_counts[source] += 1
                    if predicted[i] == labels[i]:
                        source_correct[source] += 1

        accuracy = correct / total * 100.0
        bootstrap_accuracies.append(accuracy)
        source_accuracies = {s: (source_correct[s] / source_counts[s] * 100) if source_counts[s] > 0 else 0 for s in 'ABCDEF'}
        for source in source_accuracies:
            source_bootstrap_accuracies[source].append(source_accuracies[source])

    mean_accuracy = np.mean(bootstrap_accuracies)
    std_accuracy = np.std(bootstrap_accuracies)
    mean_source_accuracies = {s: np.mean(source_bootstrap_accuracies[s]) for s in 'ABCDEF'}
    std_source_accuracies = {s: np.std(source_bootstrap_accuracies[s]) for s in 'ABCDEF'}

    return mean_accuracy, std_accuracy, mean_source_accuracies, std_source_accuracies

test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Perform bootstrap training and testing
mean_bootstrap, std_bootstrap, mean_source_accuracies, std_source_accuracies = bootstrap_train_and_test(
    best_model, custom_train_dataset_with_cluster, test_loader_loss, criterion, {'lr': best_hyperparams['lr']}, num_epochs=10, num_bootstrap=5, sample_percentage=0.8
)

print(f'Bootstrap results: Mean accuracy: {mean_bootstrap:.2f}%, Std Dev: {std_bootstrap:.2f}%')
for source in mean_source_accuracies:
    print(f'Source {source} - Bootstrap Mean: {mean_source_accuracies[source]:.2f}%, Std Dev: {std_source_accuracies[source]:.2f}%')